# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils, Statistics

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
#轉換資料
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194 
 3.299533727885655 
 2.803360380906535 
 2.9391619220655967
 2.70805020110221  
 2.9391619220655967
 3.077312260546414 
 ⋮                 
 3.1986731175506815
 3.139832617527748 
 2.9806186357439426
 2.9069010598473755
 3.054001181677967 
 2.8622008809294686
 2.8213788864092133
 3.109060958860994 
 3.0252910757955354
 3.173878458937465 
 3.091042453358316 
 2.4765384001174837

In [4]:
#訓練與測試資料集
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,9.96654,0.0,18.1,0,0.74,6.485,100.0,1.9784,24,666
2,0.01301,35.0,1.52,0,0.442,7.241,49.3,7.0379,1,284
3,0.03427,0.0,5.19,0,0.515,5.869,46.3,5.2311,5,224
4,2.14918,0.0,19.58,0,0.871,5.709,98.5,1.6232,5,403
5,0.26938,0.0,9.9,0,0.544,6.266,82.8,3.2628,4,304
6,0.06466,70.0,2.24,0,0.4,6.345,20.1,7.8278,5,358
7,0.10612,30.0,4.93,0,0.428,6.095,65.1,6.3361,6,300
8,0.26363,0.0,8.56,0,0.52,6.229,91.2,2.5451,5,384
9,3.83684,0.0,18.1,0,0.77,6.251,91.1,2.2955,24,666


In [5]:
#線性迴歸模型
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.34653      0.225479      19.2769     <1e-58   3.90324       4.78982    
Crim         -0.0104186    0.00140762    -7.40156    <1e-12  -0.013186     -0.00765121 
Chas          0.0859106    0.0400353      2.14587    0.0325   0.00720111    0.16462    
NOx          -0.848174     0.158637      -5.34665    <1e-6   -1.16005      -0.536294   
Rm            0.0720463    0.0182719      3.94301    <1e-4    0.0361237     0.107969   
Dis          -0.0467509    0.0

In [6]:
#預測
exp.(predict(model, test))

101-element Array{Float64,1}:
 15.438588909686157
 29.164598808101598
 21.11970694165178 
 16.40874481232624 
 25.142559624638658
 28.88130178737865 
 23.174267817029126
 18.396290960491708
 17.602657642055675
 29.594565150718115
 17.354416566785705
 15.397383178340489
 17.988474098315777
  ⋮                
 15.415849473963224
 18.757705963061966
 36.30935986982055 
 28.484878398739816
 17.06395758226623 
 16.870673885193693
 16.396326314576815
 12.805634090630402
 20.372369961980016
 32.60294097032114 
 20.678041406159515
 15.198187879285225

In [7]:
#評估模型
adjr²(model)

0.7665410913604531